# NeoStats — Documentation

This notebook contains the architecture, data flow, data model, assumptions, data cleaning and transformation logic, and suggested Power BI visuals for the Neostars_Project deliverable. Use this as the formal documentation artifact for the data engineering pipeline.

## 1 — Architecture (logical)

A lightweight batch pipeline designed to run locally or be mapped to Azure services. The components are:

- Data sources: Excel workbook with sheets: `Server_Metadata`, `Server_Performance_Station1`, `Server_Performance_Station2`
- Ingestion: `scripts/ingestion.py` (reads the Excel, returns DataFrames)
- Cleaning: `scripts/cleaning.py` (normalizes headers, cleans, detects anomalies)
- Transformation: `scripts/transformation.py` (computes metrics, aggregates, enriches metadata)
- Storage/Output: `../output/structured_data.csv` and `../output/aggregates_<window>.csv`
- Visualization: Power BI using the CSVs or a database sink.

ASCII diagram (logical):

```
[Excel (.xlsx)]
      |
      v
[scripts/ingestion.py] -> raw DataFrames
      |
      v
[scripts/cleaning.py] -> cleaned DataFrame + anomalies log
      |
      v
[scripts/transformation.py] -> enriched DataFrame + aggregates (1min/5min/1H)
      |
      v
[../output/*.csv] -> Power BI (PBIX)
```

## 2 — Data flow (step-by-step)

1. `ingestion.py` reads the Excel workbook using `pandas.read_excel` and returns three DataFrames: metadata, st1, st2.
2. Notebook concatenates station tables into a single performance table (`pd.concat([st1, st2])`).
3. `clean_performance_data` is called with the combined table. It performs header normalization, drops irrelevant columns, fills or imputes numeric nulls, parses timestamps and detects anomalies (missing timestamps, negative values, duplicates). It returns the cleaned DataFrame.
4. `transform_data` computes CPU and Memory utilization, attempts to compute Disk I/O and Network rates (either from provided rate columns or by differencing cumulative counters per server), enriches rows by joining with `Server_Metadata`, and writes aggregated CSVs for 1min/5min/hourly windows.
5. Final enriched dataset is saved to `../output/structured_data.csv` and can be loaded into Power BI.

## 3 — Data model (recommended tables and schemas)

Recommended table schemas (denormalized for BI):

### `servers_metadata` (from `Server_Metadata` sheet)
- server_id (PK)
- cluster
- admin_contact
- location
- os_type
- instance_size
- other_attributes...

### `server_performance` (cleaned, row-level time series)
- server_id
- timestamp (UTC)
- cpu_utilization (percent)
- memory_utilization (percent)
- disk_io_rate_mb_s
- network_in_kbps
- network_out_kbps
- anomaly_flag
- imputed_<col> flags (boolean) for traceability
- source_station

### `aggregates_<window>` (produced CSVs)
- server_id
- window_start
- avg_cpu
- p95_cpu (optional enhancement)
- avg_mem
- avg_disk_io_mb_s
- avg_net_in_kBps
- avg_net_out_kBps

These tables are written as CSV in `../output/` by the pipeline and can be swapped for Parquet/Delta/SQL when scaling to cloud storage.


## 4 — Assumptions

- The Excel file uses the sheet names described in the prompt: `Server_Metadata`, `Server_Performance_Station1`, `Server_Performance_Station2`.
- Timestamps are present or parsable; when not, rows are flagged and logged by the cleaning step.
- CPU/Memory metrics may be provided either as 'used' and 'total' columns or pre-computed percentages; the code detects both forms.
- Disk and network may be provided as instantaneous rates (MB/s, kB/s) or cumulative counters (bytes). When counters are present, the pipeline computes rates by per-server differencing and time-deltas.
- The pipeline is batch-oriented (not streaming). For production, streaming ingestion would use Kafka/EventHub + stream processing.
- Missing/invalid numeric values can be imputed with median or zero depending on policy; the cleaning function records imputation flags for traceability.


## 5 — Cleaning logic (summary)

Implemented in `scripts/cleaning.py`. Key points:

- Header normalization: the cleaner normalizes header names by removing non-alphanumerics and lowercasing, then auto-maps common variants to canonical names (e.g., `CPU Usage (%)` -> `CPU_Utilization`).
- Drop irrelevant columns: `Config_Version`, `Last_Patch_Date`, `Deployment_Token` are removed if present.
- Numeric null handling: the cleaner fills selected numeric columns with 0 where appropriate and logs the action. It also supports imputation behaviors (median/zero) in enhanced versions.
- Timestamp parsing: attempts to parse `Timestamp` or variants (e.g., `Log_Timestamp`) and coerces invalid values to NaT; these are counted/printed as anomalies.
- Negative values: detected and corrected (converted to absolute or set to NaN depending on policy).
- Duplicates: detected and dropped; duplicates are counted and printed in the anomaly summary.
- Output: a cleaned DataFrame is returned and the cleaner prints an anomaly summary suitable for audit or saved logs.


## 6 — Transformation logic (summary)

Implemented in `scripts/transformation.py`. Key points:

- CPU/Memory Utilization: computed when 'used' and 'total' columns are present, or used directly when a percentage column is available. The result is `CPU_Utilization` and `Memory_Utilization` in percent.
- Disk I/O Rate: uses an existing rate column when present, otherwise attempts to compute per-server rate by differencing cumulative counters (read/write bytes) and dividing by time delta, converted to MB/s.
- Network Throughput: similarly computed from cumulative counters (diff + time delta) or used directly when rate columns exist; results stored as `Network_In_kBps` and `Network_Out_kBps`.
- Enrichment: joined with `Server_Metadata` on `Server_ID` to add `location`, `os_type`, `instance_size`, and `cluster` for each performance row.
- Aggregations: windowed (1min, 5min, 1H) mean aggregations are produced per `Server_ID` and saved as CSVs in `../output/aggregates_<window>.csv` for use by Power BI.
- Output: the enriched performance DataFrame saved to `../output/structured_data.csv` (via notebook or utils.save_output).


## 7 — Visual insights & Power BI guidance

Suggested KPIs and visuals:

- KPI cards: Average CPU Utilization (cluster-level), % uptime (derived), Number of High-Load servers.
- Time series: CPU and memory trend per server or cluster (use aggregates_1min/5min for smoother charts).
- Heatmap / Matrix: CPU utilization by cluster vs. server to quickly identify hotspots.
- Top-N lists: Top 10 servers by average CPU or by peak memory usage over selected window.
- Drill-through: From cluster view to server detail with server metadata (OS, instance size, admin contact).
- Slicers/filters: Time window (last 24h, last 7d), Cluster, Server, OS type.

Power BI data sources: point Power BI at `../output/aggregates_hourly.csv` (or load `structured_data.csv` for row-level analysis). For better performance, switch to a parquet/SQL source in production.


## 8 — How to run the pipeline locally

Open `notebooks/neo_pipeline.ipynb` and run the cells in order, OR run the short script below in PowerShell at the project root. This will read the Excel in `./data/`, clean, transform, and write outputs to `./output/`.

```powershell
# From the project root (PowerShell)
python - <<'PY'
from scripts.ingestion import load_excel
from scripts.cleaning import clean_performance_data
from scripts.transformation import transform_data
from scripts.utils import save_output
import pandas as pd
meta, st1, st2 = load_excel('data/Data Engineering Use Case Dataset.xlsx')
perf = pd.concat([st1, st2], ignore_index=True)
cleaned = clean_performance_data(perf)
final = transform_data(cleaned, meta)
save_output(final, 'output/structured_data.csv')
print(final.head())
PY
```

If you prefer to run the notebook, the `neo_pipeline.ipynb` already includes the same steps and a logger setup.


## 9 — Artifacts produced

- `../output/structured_data.csv` — cleaned, enriched row-level dataset ready for BI.
- `../output/aggregates_1min.csv`, `../output/aggregates_5min.csv`, `../output/aggregates_hourly.csv` — aggregated time-series views.
- Power BI report (deliverable) — build on the aggregates or the row-level CSV.
- Notebook: this document (`notebooks/documentation.ipynb`) — architecture, data flow, model, and run instructions.


## 10 — Next steps and recommendations

1. Validate results against the sample dataset and inspect anomalies reported by `scripts/cleaning.py`.
2. Convert CSV aggregates to Parquet or load them into a SQL/Delta store for scalability.
3. If Azure is available: replace local steps with Azure Data Factory (ingestion), ADLS Gen2 (storage), Azure Synapse or Databricks (transformation), and Power BI Service for report publishing.
4. Add unit tests for cleaning and transformation functions, including edge cases (missing timestamps, counter rollover, duplicate detection).
5. Create a small CI pipeline to run the notebooks or tests on commit and validate data regression.

---
Documentation complete.